In [ ]:
pip install pennylane --upgrade #安裝pennylane

In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
import seaborn as sns
import jax;
jax.config.update('jax_platform_name', 'cpu')
jax.config.update("jax_enable_x64", True)
import jax.numpy as jnp
import optax  # optimization using jax
import pennylane as qml
import pennylane.numpy as pnp

In [ ]:
#公用變數
seed=0
rng=np.random.default_rng(seed=seed)
Num_wires=6
Device=qml.device("default.qubit", wires=Num_wires) #pennylane 量子電路模擬器

In [ ]:
from google.colab import drive #讓colba存取硬碟資料
drive.mount('/content/drive')

In [ ]:
def Read_MNIST(data): #從硬碟讀取資料
 fr=open("/content/drive/MyDrive/dong_final/%s_8_8.text"%(data),"r")
 img=[]
 trg=[]
 for line in fr:
  lx=line.split()
  if len(lx)==1:
   trg.append(int(lx[0]))
  else:
   for i in range (len(lx)):
    img.append(np.float64(lx[i]))
 fr.close()
 img2=np.asarray(img).reshape(-1,8*8) #將資料轉成陣列
 img3=img2/np.linalg.norm(img2,axis=1).reshape((-1,1)) #量子態平方和須為1
 return jnp.asarray(trg),jnp.asarray(img3) #轉成jnp陣列，才可以使用jax調整參數

In [ ]:
def Read_Data(): #讀取訓練和測試資料
 y_test,x_test=Read_MNIST("test")
 y_train,x_train=Read_MNIST("train")
 return x_train,y_train,x_test,y_test

In [ ]:
def convolutional_layer(weights,wires,skip_first_layer=True): #捲基層
 n_wires=len(wires)
 assert n_wires >= 3, "this circuit is too small!"
 for p in [0, 1]:
  for indx, w in enumerate(wires):
   if indx % 2 == p and indx < n_wires - 1:
    if indx % 2 == 0 and not skip_first_layer:
      qml.U3(*weights[:3],wires=[w])
      qml.U3(*weights[3:6],wires=[wires[indx+1]])
    qml.IsingXX(weights[6],wires=[w,wires[indx+1]])
    qml.IsingYY(weights[7],wires=[w,wires[indx+1]])
    qml.IsingZZ(weights[8],wires=[w,wires[indx+1]])
    qml.U3(*weights[9:12],wires=[w])
    qml.U3(*weights[12:],wires=[wires[indx+1]])

In [ ]:
def pooling_layer(weights, wires): #池化層
 n_wires = len(wires)
 assert len(wires) >= 2, "this circuit is too small!"
 for indx, w in enumerate(wires):
  if indx % 2 == 1 and indx < n_wires:
   m_outcome = qml.measure(w)
   qml.cond(m_outcome, qml.U3)(*weights, wires=wires[indx - 1])

In [ ]:
def conv_and_pooling(kernel_weights,n_wires,skip_first_layer=True): #呼叫捲基層和池化層函數
 convolutional_layer(kernel_weights[:15],n_wires,skip_first_layer=skip_first_layer)
 pooling_layer(kernel_weights[15:],n_wires)

In [ ]:
def dense_layer(weights,wires): #全連接層
 qml.ArbitraryUnitary(weights,wires)

In [ ]:
@qml.qnode(Device,interface="jax")
def conv_net(weights,last_layer_weights,features):
 layers=weights.shape[1]
 wires=list(range(Num_wires))
 qml.AmplitudeEmbedding(features=features,wires=wires,pad_with=0.5)
 qml.Barrier(wires=wires,only_visual=True)
 for j in range(layers):
  conv_and_pooling(weights[:,j],wires,skip_first_layer=(not j == 0))
  wires = wires[::2]
  qml.Barrier(wires=wires, only_visual=True)
 assert last_layer_weights.size == 4 ** (len(wires)) - 1, (
  "The size of the last layer weights vector is incorrect!"
  f" \n Expected {4 ** (len(wires)) - 1}, Given {last_layer_weights.size}")
 dense_layer(last_layer_weights, wires)
 return qml.probs(wires=(0))

In [ ]:
@jax.jit
def compute_out(weights, weights_last, features, labels):
  cost=lambda weights,weights_last,feature,label:conv_net(weights,weights_last,feature)[label]
  return jax.vmap(cost,in_axes=(None,None,0,0),out_axes=0)(weights,weights_last,features,labels)
def compute_accuracy(weights, weights_last, features, labels): #計算準確度
  out=compute_out(weights,weights_last,features,labels)
  return jnp.sum(out>0.5)/len(out)
def compute_cost(weights,weights_last,features,labels): #計算損失函數
  out=compute_out(weights,weights_last,features,labels)
  return 1.0-jnp.sum(out)/len(labels)
def init_weights(): #權重
  weights=pnp.random.normal(loc=0,scale=1,size=(18,2),requires_grad=True) #權重是可調整的
  weights_last=pnp.random.normal(loc=0,scale=1,size=4**2-1,requires_grad=True)
  return jnp.array(weights),jnp.array(weights_last)
value_and_grad=jax.jit(jax.value_and_grad(compute_cost,argnums=[0,1]))

In [ ]:
def train_qcnn(n_train,n_test,n_epochs): #量子電路主程式
 x_train,y_train,x_test,y_test=Read_Data()
 weights,weights_last=init_weights()
 cosine_decay_scheduler=optax.cosine_decay_schedule(0.1,decay_steps=n_epochs,alpha=0.95)
 optimizer=optax.adam(learning_rate=cosine_decay_scheduler)
 opt_state=optimizer.init((weights,weights_last))
 train_cost_epochs,test_cost_epochs,train_acc_epochs,test_acc_epochs=[],[],[],[]
 for step in range(n_epochs):
  train_cost,grad_circuit=value_and_grad(weights,weights_last,x_train,y_train)
  updates,opt_state=optimizer.update(grad_circuit,opt_state)
  weights,weights_last=optax.apply_updates((weights,weights_last),updates)
  train_cost_epochs.append(train_cost)
  train_acc=compute_accuracy(weights,weights_last,x_train,y_train)
  train_acc_epochs.append(train_acc)
  test_out=compute_out(weights,weights_last,x_test,y_test)
  test_acc=jnp.sum(test_out>0.5)/len(test_out)
  test_acc_epochs.append(test_acc)
  test_cost=1.0-jnp.sum(test_out)/len(test_out)
  test_cost_epochs.append(test_cost)
 return train_cost_epochs,test_cost_epochs,train_acc_epochs,test_acc_epochs

In [ ]:
def Write_acc(train_cost,test_cost,train_acc,test_acc): #輸出準確度
 fw=open("/content/drive/MyDrive/dong_final/accuracy.text","w")
 fw.write("epoch train_cost  test_cost  train_acc   test_acc\n")
 for i in range (len(train_acc)):
  fw.write("%5d %10.2f %10.2f %10.2f %10.2f\n"%(i,train_cost[i],test_cost[i],train_acc[i],test_acc[i]))
 fw.close()


In [ ]:
def Main():
 n_train=10
 n_test=10
 n_epochs=2
 train_cost,test_cost,train_acc,test_acc=train_qcnn(n_train,n_test,n_epochs)
 Write_acc(train_cost,test_cost,train_acc,test_acc)

In [ ]:
Main()